In [1]:
import importlib
import sys

In [2]:
from analysis_functions import *

In [3]:
importlib.reload(sys.modules['analysis_functions'])
from analysis_functions import *

# Загрузка данных

In [4]:
X_stress_components_new = opener('X_stress_components_new', path_import='./resourses/')
X_strain_components_new = opener('X_strain_components_new', path_import='./resourses/')
y_stress_components_new = opener('y_stress_components_new', path_import='./resourses/')
y_strain_components_new = opener('y_strain_components_new', path_import='./resourses/')
# X_stress_components_other = opener('X_stress_components_other')
# X_strain_components_other = opener('X_strain_components_other')
# y_stress_components_other = opener('y_stress_components_other')
# y_strain_components_other = opener('y_strain_components_other')

./resourses//X_stress_components_new.pkl
./resourses//X_strain_components_new.pkl
./resourses//y_stress_components_new.pkl
./resourses//y_strain_components_new.pkl


In [5]:
X_stress_components_new.shape

(3, 2678, 5)

# Определение макропараметров

In [6]:
component_num = 0
n_trials_list = [10, 50, 100, 200, 500]
n_trials_list = [10, 50, 100, ]

# Обучение BayesianOptimization

In [ ]:
best_params_list = []
best_values = []
for n_trials in n_trials_list:
    best_params, cur_X_test, cur_y_test, best_value = do_bayes_opt(
        X_stress_components_new[component_num],
        y_stress_components_new[component_num],
        n_trials=n_trials,
        n_layers=5,
        n_neurons=100,
        n_iter=10,
    )
    best_params_list.append(best_params)
    best_values.append(best_value)

|   iter    |  target   | activa... |   alpha   | early_... | learni... | learni... | max_iter  | n_layers  | n_units_0 |  solver   |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | -1.204    | 1.087     | 1.113     | 0.4245    | 1.69      | 0.0004729 | 1.832e+03 | 3.683     | 82.76     | 0.2734    |


# Обучение skopt

In [ ]:
best_params_list = []
best_values = []
for n_trials in n_trials_list:
    best_params, cur_X_test, cur_y_test, best_value = do_skopt(
        X_stress_components_new[component_num],
        y_stress_components_new[component_num],
        n_trials=n_trials,
        n_layers=2,
        n_neurons=10,
        n_calls=10
    )
    best_params_list.append(best_params)
    best_values.append(best_value)

/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1157) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1157) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1157) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs fa

Best Hyperparameters: {'n_layers': np.int64(2), 'n_units_0': np.int64(7), 'learning_rate_init': 4.977392799783211e-06, 'early_stopping': True, 'max_iter': np.int64(10392), 'learning_rate': 'adaptive', 'alpha': 0.1, 'activation': 'relu', 'solver': 'lbfgs'}


/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/s

# Обучение optuna

In [ ]:
best_params_list = []
best_values = []
for n_trials in n_trials_list:
    best_params, cur_X_test, cur_y_test, best_value = do_optuna(
        X_stress_components_new[component_num],
        y_stress_components_new[component_num],
        n_trials=n_trials,
        n_layers=5,
        n_neurons=100,
    )
    best_params_list.append(best_params)
    best_values.append(best_value)

[I 2025-01-14 22:06:31,107] A new study created in memory with name: no-name-11e8501d-ec49-400b-8ca5-0f7c03382790


# Сохранение моделей

In [ ]:
path_to_save_model = './learned_models/'
test_metrics = []
for i, best_params in enumerate(best_params_list):
    optuna_200_3cv, cur_prediction, cur_y_test, cur_X_test, test_metric = test_after_opt(
        best_params, X_stress_components[component_num], y_stress_components[component_num],
        f'optuna_{n_trials_list[i]}_3cv', path_to_save_model)
    test_metrics.append(test_metric)

In [ ]:
sns.set_style('whitegrid')
plt.plot(n_trials_list, test_metrics, '-o')

## Ячейка для запуска кастомной модели

In [8]:
component_num = 2
path_to_save_model = "./learned_models/"
best_params = {
    "n_layers": 4,
    "n_units_0": 87,
    "n_units_1": 73,
    "n_units_2": 42,
    "n_units_3": 69,
    "learning_rate_init": 5.906544120294311e-05,
    "early_stopping": True,
    "max_iter": 11240,
    "learning_rate": "invscaling",
    "alpha": 0.0001,
    "activation": "tanh",
    "solver": "lbfgs",
}


name = "component_2"
optuna_200_3cv, cur_prediction, cur_y_test, cur_X_test, test_metric = test_after_opt(
    best_params,
    X_stress_components_new[component_num],
    y_stress_components_new[component_num],
    name,
    path_to_save_model,
)

/Users/i.grebenkin/.local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


saved in ./learned_models//component_2.pkl
test rmse = 1.182551894890011


ValueError: too many values to unpack (expected 5)

In [19]:
component_num = 1
path_to_save_model = "./learned_models/"
best_params = {
    "n_layers": 2,
    "n_units_0": 65,
    "n_units_1": 92,
    "learning_rate_init": 0.003016144421699664,
    "early_stopping": True,
    "max_iter": 9546,
    "learning_rate": "adaptive",
    "alpha": 0.001,
    "activation": "tanh",
    "solver": "sgd",
}


name = f"component_{component_num}"
best_regr, cur_prediction, cur_y_test, cur_X_test, test_error, train_error = test_after_opt(
    best_params,
    X_stress_components_new[component_num],
    y_stress_components_new[component_num],
    name,
    path_to_save_model,
)

saved in ./learned_models//component_1.pkl
test rmse = 6.277783867516113


In [9]:
component_num = 0
path_to_save_model = "./learned_models/"
best_params = {
    "n_layers": 5,
    "n_units_0": 39,
    "n_units_1": 30,
    "n_units_2": 31,
    "n_units_3": 32,
    "n_units_4": 32,
    "learning_rate_init": 2.1155868863441607e-06,
    "early_stopping": False,
    "max_iter": 14151,
    "learning_rate": "invscaling",
    "alpha": 0.001,
    "activation": "relu",
    "solver": "lbfgs",
}


name = "component_0"
best_regr, cur_prediction, cur_y_test, cur_X_test, test_error, train_error = test_after_opt(
    best_params,
    X_stress_components_new[component_num],
    y_stress_components_new[component_num],
    name,
    path_to_save_model,
)

saved in ./learned_models//component_0.pkl
test rmse = 0.852824487605159


/Users/i.grebenkin/.local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [22]:
import shutil


def plot_results(
    x,
    y,
    model_name,
    do_zip=False,
    dpi=500,
    delete_and_rewrite=True,
    _path_import=path_to_save_model,
):
    cur_X_test, cur_y_test, *_ = get_train_test(x, y)

    # (best_regr, cur_prediction, cur_y_test, cur_X_test)
    model = opener(model_name, path_import=_path_import)[0]
    cur_prediction = model.predict(cur_X_test)
    print("rmse = ", np.sqrt(mean_squared_error(cur_y_test, cur_prediction)))

    if delete_and_rewrite:
        os.makedirs(model_name, exist_ok=True)

    for i, item in enumerate(cur_X_test[::20]):
        fig = plt.figure(figsize=(5, 4))
        plt.plot(
            cur_X_test[i * 20 : (i + 1) * 20, -1],
            cur_prediction[i * 20 : (i + 1) * 20],
            "-o",
            label="predicted",
        )
        plt.plot(
            cur_X_test[i * 20 : (i + 1) * 20, -1],
            cur_y_test[i * 20 : (i + 1) * 20],
            "-o",
            label="real",
        )
        title = f"red {item[0]}, cal {item[1]}, alpha {int(item[2])}, fric {item[3]} vel {item[4]}"
        plt.title(title)
        plt.legend(loc="best")
        plt.xlabel("r, -")
        plt.ylabel("$\sigma$, MPa")
        # plt.show()
        plt.savefig("".join((model_name, "/", title, ".png")), format="png", dpi=dpi)
        plt.close()
        # break
    if do_zip:
        shutil.make_archive(model_name, "zip", model_name)

In [21]:
X_stress_components_new[0].max(axis=0)

array([ 0.25,  1.  , 20.  ,  0.1 , 40.  ])

In [25]:
plot_results(
    X_stress_components_new[0],
    y_stress_components_new[0],
    'component_0',
    do_zip=0,
    _path_import=path_to_save_model,
    delete_and_rewrite=True
)

./learned_models//component_0.pkl
rmse =  0.852824487605159


In [26]:
component_num = 1
plot_results(
    X_stress_components_new[1],
    y_stress_components_new[1],
    f'component_{component_num}',
    do_zip=0,
    _path_import=path_to_save_model
)

./learned_models//component_1.pkl
rmse =  6.277783867516113


In [27]:
plot_results(
    X_stress_components_new[2],
    y_stress_components_new[2],
    'component_2',
    do_zip=0,
    _path_import=path_to_save_model
)

./learned_models//component_2.pkl
rmse =  1.182551894890011


In [98]:
plot_results(
    X_stress_components[0],
    y_stress_components[0],
    'optuna_200_3cv',
    do_zip=True
)

d:\учеба/optuna_200_3cv.pkl
rmse =  4.872450206028714
